In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

##### Input size = [3, 385, 1085]

In [2]:
def swish(x):
    return x/(1+torch.exp(-x))

# Custom 2d average pooling. Used directly prior to the fully connected layer(s)
#
# dims: Tuple representing the two dimensions to pool.
# keep_dims: Boolean determining if output should preserve number of input dimensions.
def avg_pool(x, dims, keep_dims=False):
    summed_tensor = torch.sum(x, dim=dims)
    if keep_dims:
        reshaped_tensor = summed_tensor.unsqueeze(dim=dims[0]).unsqueeze(dim=dims[1])
        return reshaped_tensor
    return summed_tensor

In [3]:
# MobileNET arch
class SingNET(nn.Module):
    def __init__(self):
        super(SingNET, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu = nn.ReLU()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(32, 32, kernel_size=3, stride=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            
            nn.Conv2d(32, 32, kernel_size=1, stride=1),
            nn.BatchNorm2d(32),
            nn.ReLU()
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 32, kernel_size=3, stride=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            
            nn.Conv2d(32, 64, kernel_size=1, stride=1),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            
            nn.Conv2d(64, 64, kernel_size=1, stride=1),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        
        self.layer4 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, stride=2),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            
            nn.Conv2d(64, 64, kernel_size=1, stride=1),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        
        self.layer5 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            
            nn.Conv2d(64, 128, kernel_size=1, stride=1),
            nn.BatchNorm2d(128),
            nn.ReLU()
        )
        
        self.layer6 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, stride=2),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            
            nn.Conv2d(128, 128, kernel_size=1, stride=1),
            nn.BatchNorm2d(128),
            nn.ReLU()
        )
        
        self.layer7_12 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, stride=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            
            nn.Conv2d(128, 128, kernel_size=1, stride=1),
            nn.BatchNorm2d(128),
            nn.ReLU()
        )
        
        self.layer13 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, stride=2),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            
            nn.Conv2d(128, 256, kernel_size=1, stride=1),
            nn.BatchNorm2d(256),
            nn.ReLU()
        )
        
        self.layer14 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=2),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            
            nn.Conv2d(256, 512, kernel_size=1, stride=1),
            nn.BatchNorm2d(512),
            nn.ReLU()
        )
        
        self.fc = nn.Linear(512, 5)
        
        
        
    def forward(self, x):
        out = self.bn1(self.conv1(x))
        out = self.relu(out)
        
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = self.layer7_12(out)
        out = self.layer7_12(out)
        out = self.layer7_12(out)
        out = self.layer7_12(out)
        out = self.layer7_12(out)
        out = self.layer7_12(out)
        out = self.layer13(out)
        out = self.layer14(out)
        
        out = avg_pool(out, (2,3))
        out = self.fc(out)
        return out
        
        

In [4]:
img = Image.open("data/costa_rica/spectrogram/Broad-billed_Motmot/Broad-billed_Motmot_105_0.jpg")
transform = transforms.Compose([
    transforms.ToTensor()
])

img_tensor = transform(img).unsqueeze(0)

In [5]:
img_tensor.size()

torch.Size([1, 3, 385, 1085])

In [6]:
net = SingNET()

net.forward(img_tensor)

tensor([[-2.9098, -0.4664, -0.2132, -3.4092,  3.8125]],
       grad_fn=<AddmmBackward0>)

In [7]:
num_params = sum(p.numel() for p in net.parameters())
print(num_params)

1387589


In [8]:
def train(net, epochs=100, batch_size=8, lr=0.005, L2_reg=0.01, saved_filepath="saved_models/test_model.pt"):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Device in use: {device}")
    if device == "cuda":
        torch.cuda.empty_cache()
        #torch.cuda.max_memory_allocated(max_split_size_mb=1024)
    net.to(device)
    print("==> Starting Data Preparation...")
    data_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])
    ])
    train_set = ImageFolder("data/costa_rica/train", transform=data_transform)
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=1)
    val_set = ImageFolder("data/costa_rica/validation", transform=data_transform)
    val_loader = DataLoader(val_set, batch_size=1, shuffle=False, num_workers=1)
    
    loss_function = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=lr, weight_decay=L2_reg)
    #scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[int(epochs*0.5), int(epochs*0.75)], gamma=0.1)
    best_acc = 0
    
    print("==> Training Initiated...")
    for epoch in range(0, epochs):
        print(f"Epoch {epoch}/{epochs}")
        net.train()
        train_loss = 0
        correct = 0
        total = 0
        
        for batch_idx, (inputs, targets) in enumerate(train_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = net.forward(inputs)
            loss = loss_function(outputs, targets)
            loss.backward()
            
            optimizer.step()
            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            
        print(f"Train Loss: {round(train_loss,3)} | Train Acc: {round(correct/total,3)}")
        #scheduler.step()
        
        net.eval()
        val_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            for batch_idx, (inputs, targets) in enumerate(val_loader):
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = net.forward(inputs)
                loss = loss_function(outputs, targets)
                
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                total += targets.size(0)
                correct += predicted.eq(targets).sum().item()
        val_acc = correct/total
        
        print(f"Val Loss:   {round(val_loss,3)} | Val Acc:   {round(val_acc,3)}")
        
        if val_acc > best_acc:
            best_acc = val_acc
            print("Saving Model...")
            torch.save(net.state_dict(), saved_filepath)
    
    print("=========> Training Complete <=========")
    print(f"Best Validation Accuracy: {best_acc}")
            
            
    

In [9]:
net = SingNET()
train(net)

Device in use: cuda
==> Starting Data Preparation...
==> Training Initiated...
Epoch 0/100
Train Loss: 445.40128153562546 | Train Acc: 0.3524416135881104
Val Loss:   715.9112917482853 | Val Acc:   0.375
Saving Model...
Epoch 1/100
Train Loss: 296.87821167707443 | Train Acc: 0.3913658881811748
Val Loss:   216.21432462288067 | Val Acc:   0.4375
Saving Model...
Epoch 2/100
Train Loss: 243.63993626832962 | Train Acc: 0.44019815994338285
Val Loss:   220.17544424533844 | Val Acc:   0.4375
Epoch 3/100
Train Loss: 187.86348417401314 | Train Acc: 0.5746638358103326
Val Loss:   291.85523676872253 | Val Acc:   0.1625
Epoch 4/100
Train Loss: 162.0486858189106 | Train Acc: 0.6178343949044586
Val Loss:   264.0709728002548 | Val Acc:   0.4375
Epoch 5/100
Train Loss: 152.70200729370117 | Train Acc: 0.640481245576787
Val Loss:   290.76832419633865 | Val Acc:   0.4375
Epoch 6/100
Train Loss: 160.31395837664604 | Train Acc: 0.6242038216560509
Val Loss:   264.84085220098495 | Val Acc:   0.4375
Epoch 7/100

KeyboardInterrupt: 